In [ ]:


import csv
from pathlib import Path

import cv2
import imageio
import imageio_ffmpeg
import numpy as np
import onnxruntime as ort
import torch
import os
import torchvision.transforms as T
from PIL import Image
from sparrow_datums import BoxTracking
from tqdm import tqdm
from torch.utils.data import DataLoader
from speed_trapv3.keypoints.dataset import SegmentationDataset
from speed_trapv3.keypoints.utils import Holdout
import matplotlib.pyplot as plt
from speed_trapv3.keypoints.model import SegmentationModel
from speed_trapv3.keypoints.config import Config
import matplotlib.pyplot as plt

data = SegmentationDataset(Holdout.DEV)
dataloader = DataLoader(data, batch_size=1, shuffle=False)
model = SegmentationModel().eval().cuda()
model.load(Config.trained_model_path)
for batch in iter(dataloader):
    x = batch['image']
    x = x.cuda()
    result = model(x)
    keypoints = result["keypoints"][0].detach().cpu().numpy()
    # keypoints = batch['keypoints'].detach().cpu().numpy()[0]
    transform = T.ToPILImage()
    img = np.array(transform(x[0]))
    img = cv2.circle(img, (int(keypoints[0][0]), int(keypoints[0][1])), radius=10, color=(0, 0, 255), thickness=-1) #Blue: Backtire
    img = cv2.circle(img, (int(keypoints[1][0]), int(keypoints[1][1])), radius=10, color=(255, 0, 0), thickness=-1) #Red Front tire
    save_path = '/code/data/datasets/temp_imgs'
    filename = batch['annotation_path'][0].split("/")[-1] + ".jpg"
    cv2.imwrite(os.path.join(save_path, filename), img)